In [1]:
import pandas as pd
import investos as inv
from investos.portfolio.cost_model import *
from investos.portfolio.constraint_model import *

In [2]:
actual_returns = pd.read_parquet("https://investos.io/example_actual_returns.parquet")
forecast_returns = pd.read_parquet("https://investos.io/example_forecast_returns.parquet")

In [3]:
# For trading costs:
actual_prices = pd.read_parquet("https://investos.io/example_spo_actual_prices.parquet")
forecast_volume = pd.Series(
    pd.read_csv("https://investos.io/example_spo_forecast_volume.csv", index_col="asset")
    .squeeze(),
    name="forecast_volume"
)
forecast_std_dev = pd.Series(
    pd.read_csv("https://investos.io/example_spo_forecast_std_dev.csv", index_col="asset")
    .squeeze(),
    name="forecast_std_dev"
)
half_spread_percent = 2.5 / 10_000 # 2.5 bps
half_spread = pd.Series(index=forecast_returns.columns, data=half_spread_percent)

In [4]:
# For short holding costs:
short_cost_percent = 40 / 10_000 # 40 bps
trading_days_per_year = 252
short_rates = pd.Series(index=forecast_returns.columns, data=short_cost_percent/trading_days_per_year)

In [5]:
strategy = inv.portfolio.strategy.SPO(
    actual_returns = actual_returns,
    forecast_returns = forecast_returns,
    costs = [
        ShortHoldingCost(short_rates=short_rates, exclude_assets=["cash"]),
        TradingCost(
            actual_prices=actual_prices,
            forecast_volume=forecast_volume,
            forecast_std_dev=forecast_std_dev,
            half_spread=half_spread,
            exclude_assets=["cash"],
        ),
    ],
    constraints = [
        MaxShortLeverageConstraint(limit=0.3),
        MaxLongLeverageConstraint(limit=1.3),
        MinWeightConstraint(limit=-0.03),
        MaxWeightConstraint(limit=0.03),
        LongCashConstraint(),
        MaxAbsTurnoverConstraint(limit=0.05),
    ],
    cash_column_name="cash"
)

In [6]:
portfolio = inv.portfolio.BacktestController(
    strategy=strategy,
    start_date='2017-01-01',
    end_date='2018-01-01',
    hooks = {
        "after_trades": [
            lambda backtest, t, u, h_next: print(".", end=''),
        ]
    }
)

In [7]:
backtest_result = portfolio.generate_positions()

Generating historical portfolio trades and positions at 2025-09-09 20:13:44...
...........................................................................................................................................................................................................................................................

Done simulating at 2025-09-09 20:15:17.


In [8]:
backtest_result.summary

Initial timestamp                         2017-01-03 00:00:00
Final timestamp                           2017-12-29 00:00:00
Total portfolio return (%)                              4.29%
Annualized portfolio return (%)                         4.35%
Annualized excess portfolio return (%)                  1.29%
Annualized excess risk (%)                              2.52%
Information ratio (x)                                   0.51x
Annualized risk over risk-free (%)                      2.52%
Sharpe ratio (x)                                        0.51x
Max drawdown (%)                                        1.11%
Annual turnover (x)                                     12.8x
Portfolio hit rate (%)                                  59.2%
